# Max Flow Applications

The purpose of this assignment is to investigate applications of finding a Max Flow. The problem asks you to design and implement an algorithm for shipping a material between nodes with different supply and demand requirements.

## Movie distribution

First solve Problem 3 from HW3-theoretical. 

Now suppose a movie distributor would like to ship a copy of a film from CA to every other state. There are therefore 48 units to ship out of CA, and each other state receives 1 unit. 

The dataset contiguous-usa.dat lists the adjacent states in the US. Each line lists two adjacent states; thus AK and HI are omitted, but DC is included in the data. The following code reads in the graph of US states.

In [1]:
import networkx as nx
G = nx.Graph()

usa = open('contiguous-usa.dat')
for line in usa:
    s1, s2 = line.strip().split()
    G.add_edge(s1, s2)

We now encode the demands into the graph.

In [2]:
for state in G.nodes():
    if state != 'CA':
        G.node[state]['demand'] = 1
G.node['CA']['demand'] = -48

We will assign a uniform capacity of 16 to each edge. Since CA has only three adjacent states, this is the smallest possible uniform capacity that allows one to ship all 48 units out of CA. As we have created an undirected graph, and flows have directions, we first convert the graph to a directed graph.

In [3]:
G = nx.DiGraph(G)
uniform_capacity = 16
for (s1, s2) in G.edges():
    G.edge[s1][s2]['capacity'] = uniform_capacity

Complete the following function to implement your algorithm to find a flow with demands. Your function should work correctly for any input, not just the movie instance considered here. As always, you are encouraged to define auxiliary functions as needed for clarity.

In [4]:
def flow_with_demands(graph):
    graph.add_node('S')
    graph.add_node('T')
    graph.node['S']['demand']=0
    graph.node['T']['demand']=0
    for i in G.nodes():
        if(graph.node[i]['demand']<0):
            graph.add_edge('S', i)
            graph.edge['S'][i]['capacity'] = -graph.node[i]['demand']
        elif(G.node[i]['demand']>0):
            graph.add_edge(i, 'T')
            graph.edge[i]['T']['capacity'] = graph.node[i]['demand']
    if nx.maximum_flow(graph, 'S', 'T')[0] < sum(graph.node[i]['demand'] for i in graph.nodes() if graph.node[i]['demand']>0):
        raise nx.NetworkXUnfeasible('NetworkXUnfeasible: No flow satisfying the demands!')
    flow = nx.maximum_flow(graph, 'S', 'T')[1]
    flow.pop('S')
    flow.pop('T')
    for i in flow:
        try:
            flow[i].pop('S')
        except:
            pass
        try:
            flow[i].pop('T')
        except:
            continue
    graph.node.pop('S')
    graph.node.pop('T')
    return flow

To verify that your solution is correct, implement a function that computes the total flow into each node (which will be negative for supply nodes).

In [5]:
def divergence(flow):
    res = {}
    for i in flow:
        if i not in res:
            res[i] = -(sum(flow[i].values()))
        else:
            res[i] -= sum(flow[i].values())
        for j in flow[i]:
            if j not in res:
                res[j] = flow[i][j]
            else:
                res[j] += flow[i][j]
    return res

The following code performs a sanity check on your function (but does not completely confirm correctness).

In [6]:
flow = flow_with_demands(G)
div = divergence(flow)
print "Flow satisfies all demands:", all(div[n] == G.node[n]['demand'] for n in G.nodes())

Flow satisfies all demands: True
